In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langsmith import traceable
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA


In [2]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = TextLoader("../files/chapter_three.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = Chroma.from_documents(docs, cached_embeddings)


In [3]:
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
)


In [4]:
chain.invoke("Aaronson 은 유죄인가요?")


{'query': 'Aaronson 은 유죄인가요?', 'result': '죄송합니다. 그 질문에 대한 답을 모르겠습니다.'}

In [5]:
chain.invoke("그가 테이블에 어떤 메시지를 썼나요?")

{'query': '그가 테이블에 어떤 메시지를 썼나요?',
 'result': '그는 먼지투성이 테이블에 "2 + 2 = 5"라고 썼습니다.\n'}

In [6]:
chain.invoke("Julia 는 누구인가요?")


{'query': 'Julia 는 누구인가요?',
 'result': '제공된 텍스트에서 줄리아는 윈스턴과 관계를 맺었던 인물입니다.  그녀는 윈스턴이 사랑했고, 그와 함께 반란을 꿈꿨지만, 결국 윈스턴은 그녀를 배신합니다.  자세한 관계나 성격은 텍스트에 완전히 드러나 있지 않습니다.\n'}